In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from svea_expvis.config import Config
from svea_expvis.readers.txt import text_reader
from svea_expvis.bokeh_widgets.map import get_map
from svea_expvis.bokeh_widgets.callbacks import lasso_corr_callback, select_callback, update_colormapper
from svea_expvis.bokeh_widgets.utils import get_columndata_source

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PreText, Select, Circle
from bokeh.layouts import column, row


DEFAULT_PARAMETERS = ['TEMP_BTL', 'TEMP_CTD', 'SALT_BTL', 'SALT_CTD', 'DOXY_BTL', 'DOXY_CTD', 'FLUO_CTD', 'CPHL']

cfig = Config(data_file_name='phyche_archive.feather')

output_notebook()


Loading BokehJS ...

In [2]:
df = pd.read_feather(cfig.data_path)
# df
position_source = get_columndata_source(
    df[['STATN', 'LATIT', 'LONGI', 'KEY', 'timestamp']].drop_duplicates().reset_index(drop=True), 
    lat_col='LATIT', 
    lon_col='LONGI',
)
data_source = get_columndata_source(df[['KEY', 'DEPH'] + DEFAULT_PARAMETERS], 'FLUO_CTD', 'CPHL')

..\svea_expvis\bokeh_widgets\utils.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'] = df[params[0]]
..\svea_expvis\bokeh_widgets\utils.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df[params[1]]


In [3]:
fig_map = get_map()

map_renderer = fig_map.circle(
    x="LONGI", 
    y="LATIT",
    size=10,
    alpha=0.5,
    source=position_source,
)
nonselected_circle = Circle(fill_alpha=0.3, line_color='grey')
map_renderer.nonselection_glyph = nonselected_circle


In [4]:
corr = figure(
    width=400, height=400,
    title='Selected X- and Y values',
    tools='pan,wheel_zoom,box_select,reset,save',
)
corr.xaxis.axis_label = 'FLUO_CTD'
corr.yaxis.axis_label = 'CPHL'
corr_source = ColumnDataSource(data=dict(x=[], y=[]))
corr.circle(
    x="x", 
    y="y",
    size=3,
    alpha=0.5,
    source=corr_source,
)

GlyphRenderer(id='1088', ...)

In [5]:
x_sel = Select(value='FLUO_CTD', options=DEFAULT_PARAMETERS, title='X-parameter')
y_sel = Select(value='CPHL', options=DEFAULT_PARAMETERS, title='Y-parameter')

x_sel.js_on_change("value", select_callback(data_source=data_source, axis_obj=corr.xaxis[0], axis='x'))
y_sel.js_on_change("value", select_callback(data_source=data_source, axis_obj=corr.yaxis[0], axis='y'))

In [6]:
selection_lasso_change = lasso_corr_callback(
     x_selector=x_sel, 
     y_selector=y_sel, 
     data_source=data_source, 
     position_source=position_source,
     corr_source=corr_source
)
position_source.selected.js_on_change('indices', selection_lasso_change)

NameError: name 'lasso_corr_callback' is not defined

In [8]:
# show(row(column(fig_map, ts1, ts2), column(corr, x_sel, y_sel)))
show(row(column(fig_map), column(corr, x_sel, y_sel)))